In [1]:
import cv2
import numpy as np
import os
from typing import Tuple, Optional

class ImageProcessor:
    def __init__(self):
        """Initialize the image processor with default parameters"""
        self.supported_formats = ('.jpg', '.jpeg', '.png', '.tif', '.tiff')

    def load_image(self, image_path: str) -> Optional[np.ndarray]:
        """
        Load an image from file with error handling

        Args:
            image_path (str): Path to the image file

        Returns:
            np.ndarray: Loaded image or None if error occurs
        """
        if not os.path.isfile(image_path):
            print(f"Error: File not found at {image_path}")
            return None

        if not image_path.lower().endswith(self.supported_formats):
            print(f"Error: Unsupported file format. Supported formats are {self.supported_formats}")
            return None

        try:
            image = cv2.imread(image_path)
            if image is None:
                raise ValueError("File could not be read as an image")
            return image
        except Exception as e:
            print(f"Error loading image: {str(e)}")
            return None

    def adjust_brightness_contrast(self, image: np.ndarray, alpha: float = 1.0, beta: int = 0) -> np.ndarray:
        """
        Adjust image brightness and contrast

        Args:
            image: Input image
            alpha: Contrast control (1.0 means no change)
            beta: Brightness control (0 means no change)

        Returns:
            Adjusted image
        """
        return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

    def apply_gaussian_blur(self, image: np.ndarray, kernel_size: Tuple[int, int] = (5, 5),
                          sigma_x: int = 0) -> np.ndarray:
        """
        Apply Gaussian blur to the image

        Args:
            image: Input image
            kernel_size: Size of the Gaussian kernel (width, height)
            sigma_x: Gaussian kernel standard deviation in X direction

        Returns:
            Blurred image
        """
        return cv2.GaussianBlur(image, kernel_size, sigma_x)

    def apply_canny_edge(self, image: np.ndarray, threshold1: int = 100,
                        threshold2: int = 200) -> np.ndarray:
        """
        Apply Canny edge detection

        Args:
            image: Input image (should be grayscale)
            threshold1: First threshold for hysteresis procedure
            threshold2: Second threshold for hysteresis procedure

        Returns:
            Image with edges detected
        """
        if len(image.shape) > 2:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image
        return cv2.Canny(gray, threshold1, threshold2)

    def apply_denoising(self, image: np.ndarray, h: float = 10,
                       template_window_size: int = 7,
                       search_window_size: int = 21) -> np.ndarray:
        """
        Apply Non-local Means Denoising

        Args:
            image: Input image
            h: Parameter regulating filter strength
            template_window_size: Size in pixels of the template patch
            search_window_size: Size in pixels of the window used to compute weighted average

        Returns:
            Denoised image
        """
        return cv2.fastNlMeansDenoisingColored(image, None, h, h, template_window_size, search_window_size)

    def extract_features(self, image: np.ndarray) -> np.ndarray:
        """
        Extract basic features (SIFT descriptors) from the image

        Args:
            image: Input image

        Returns:
            Keypoints and descriptors
        """
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        sift = cv2.SIFT_create()
        keypoints, descriptors = sift.detectAndCompute(gray, None)
        return descriptors

    def save_image(self, image: np.ndarray, output_path: str) -> bool:
        """
        Save image to file with meaningful name

        Args:
            image: Image to save
            output_path: Path to save the image

        Returns:
            bool: True if successful, False otherwise
        """
        try:
            cv2.imwrite(output_path, image)
            return True
        except Exception as e:
            print(f"Error saving image: {str(e)}")
            return False

def main():
    """Main function to demonstrate the image processing pipeline"""
    processor = ImageProcessor()

    # Load an example image
    input_path = "/content/images (1).jpeg"
    image = processor.load_image(input_path)
    if image is None:
        return

    # Process the image through all steps
    # 1. Brightness/Contrast adjustment
    adjusted = processor.adjust_brightness_contrast(image, alpha=1.2, beta=10)

    # 2. Gaussian blur
    blurred = processor.apply_gaussian_blur(adjusted, kernel_size=(7,7))

    # 3. Denoising
    denoised = processor.apply_denoising(blurred)

    # 4. Edge detection
    edges = processor.apply_canny_edge(denoised)

    # 5. Feature extraction
    features = processor.extract_features(denoised)
    print(f"Extracted {len(features)} features")

    # Save all processed images
    base_name = os.path.splitext(os.path.basename(input_path))[0]
    processor.save_image(adjusted, f"{base_name}_adjusted.jpg")
    processor.save_image(blurred, f"{base_name}_blurred.jpg")
    processor.save_image(denoised, f"{base_name}_denoised.jpg")
    processor.save_image(edges, f"{base_name}_edges.jpg")

    print("Image processing completed successfully")

if __name__ == "__main__":
    main()

Extracted 38 features
Image processing completed successfully
